<h3 style="text-align: center; font-family: Arial, sans-serif; color: #4CAF50;">TSS - Temporal Similarity Search</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Charger les données
df = pd.read_csv("../Datasources/MetroPT3_imputed_final.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)

# Convertir la colonne timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Sélection des colonnes continues et catégoriques
continuous_features = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]
categorical_features = ["COMP", "DV_eletric", "Towers", "LPS", "Pressure_switch", "Oil_level", "Caudal_impulses"]
columns_to_keep = ["timestamp", "panne"] + continuous_features + categorical_features

# Filtrer les colonnes utiles
df = df[columns_to_keep]


In [10]:
display(df.head(3))

,timestamp,TP2,DV_pressure,Oil_temperature,Motor_current,Reservoirs,COMP,DV_eletric,Towers,LPS,Pressure_switch,Oil_level,Caudal_impulses,panne
0,2020-02-01 00:00:00,-0.012,-0.024,53.600,0.0400,9.358,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
1,2020-02-01 00:00:10,-0.014,-0.022,53.675,0.0400,9.348,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
2,2020-02-01 00:00:20,-0.012,-0.022,53.600,0.0425,9.338,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0


In [2]:
# Supprimer la colonne 'panne' de df
df = df.drop(columns=['panne'])

# Vérifier si la colonne a été supprimée
display(df.head())


,timestamp,TP2,DV_pressure,Oil_temperature,Motor_current,Reservoirs,COMP,DV_eletric,Towers,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,2020-02-01 00:00:00,-0.012,-0.024,53.600,0.0400,9.358,1.0,0.0,1.0,0.0,1.0,1.0,1.0
1,2020-02-01 00:00:10,-0.014,-0.022,53.675,0.0400,9.348,1.0,0.0,1.0,0.0,1.0,1.0,1.0
2,2020-02-01 00:00:20,-0.012,-0.022,53.600,0.0425,9.338,1.0,0.0,1.0,0.0,1.0,1.0,1.0
3,2020-02-01 00:00:30,-0.012,-0.022,53.425,0.0400,9.328,1.0,0.0,1.0,0.0,1.0,1.0,1.0
4,2020-02-01 00:00:40,-0.012,-0.022,53.475,0.0400,9.318,1.0,0.0,1.0,0.0,1.0,1.0,1.0


In [3]:
# Ajouter les classes de panne
# Classe 0 : Pas de panne détectée
# Classe 1 : En pleine panne
# Classe 2 : Panne prévue dans moins de 30 minutes

# Liste des intervalles de pannes
pannes = [
    {'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'start': '2020-05-29 23:30:00', 'end': '2020-05-30 06:00:00'},
    {'start': '2020-06-05 10:00:00', 'end': '2020-06-07 14:30:00'},
    {'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
         ]

# Convertir les timestamps des pannes en datetime
for panne in pannes:
    panne['start'] = pd.to_datetime(panne['start'])
    panne['end']   = pd.to_datetime(panne['end'])

# Ajouter une colonne 'panne' avec la valeur par défaut 0 (aucune panne détectée)
df['panne'] = 0


for panne in pannes:
    # Classe 1 : En pleine panne
    df.loc[(df['timestamp'] >= panne['start']) & (df['timestamp'] <= panne['end']), 'panne'] = 1
    # Classe 2 : Panne prévue dans moins de 15 minutes
    df.loc[(df['timestamp'] < panne['start']) & (df['timestamp'] >= panne['start'] - pd.Timedelta(minutes=15)), 'panne'] = 2

display(df.head(50))

,timestamp,TP2,DV_pressure,Oil_temperature,Motor_current,Reservoirs,COMP,DV_eletric,Towers,LPS,Pressure_switch,Oil_level,Caudal_impulses,panne
0,2020-02-01 00:00:00,-0.012,-0.024,53.600,0.0400,9.358,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
1,2020-02-01 00:00:10,-0.014,-0.022,53.675,0.0400,9.348,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
2,2020-02-01 00:00:20,-0.012,-0.022,53.600,0.0425,9.338,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
3,2020-02-01 00:00:30,-0.012,-0.022,53.425,0.0400,9.328,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
4,2020-02-01 00:00:40,-0.012,-0.022,53.475,0.0400,9.318,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
5,2020-02-01 00:00:50,-0.012,-0.024,53.500,0.0400,9.308,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
6,2020-02-01 00:01:00,-0.012,-0.024,53.375,0.0400,9.298,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
7,2020-02-01 00:01:10,-0.014,-0.024,53.550,0.0400,9.286,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
8,2020-02-01 00:01:20,-0.012,-0.022,53.425,0.0400,9.276,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0
9,2020-02-01 00:01:30,-0.012,-0.022,53.375,0.0400,9.264,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0


In [4]:
###################################################################################################
################ Train : Panne1   (1J ET DEMI)                                              #######
################ Test  : Panne4  (15mn + 15mn + 15mn)     #########################################
###################################################################################################

# Classe 0 : Pas de panne détectée
# Classe 1 : En pleine panne
# Classe 2 : Panne prévue dans moins de 30 minutes

pannes = [
    {'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'start': '2020-05-29 23:30:00', 'end': '2020-05-30 06:00:00'},
    {'start': '2020-06-05 10:00:00', 'end': '2020-06-07 14:30:00'},
    {'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
         ]


# Définir les périodes d'entraînement et de test
train_periods = [{'start': '2020-02-02 00:00', 'end': '2020-06-07 14:30:00'}]
test_periods  = [{'start': '2020-06-07 14:30:10', 'end': '2020-07-15 19:00:00'}]

# Séparer les données pour l'entraînement
train_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in train_periods
])

# Séparer les données pour le test
test_df = pd.concat([
    df[(df['timestamp'] >= pd.Timestamp(period['start'])) & 
       (df['timestamp'] <= pd.Timestamp(period['end']))]
    for period in test_periods
])

print(f"Entrainement : {len(train_df)} lignes")
print(f"Test : {len(test_df)} lignes")

Entrainement : 1093861 lignes
Test : 329940 lignes


In [11]:
# Distribution des modalités de la variable 'panne' dans l'ensemble d'entraînement
print("Modalités de 'panne' dans l'ensemble d'entraînement :")
print(train_df["panne"].value_counts())

# Distribution des modalités de la variable 'panne' dans l'ensemble de test
print("\nModalités de 'panne' dans l'ensemble de test :")
print(test_df["panne"].value_counts())


Modalités de 'panne' dans l'ensemble d'entraînement :
0    1063714
1      29877
2        270
Name: panne, dtype: int64

Modalités de 'panne' dans l'ensemble de test :
0    328229
1      1621
2        90
Name: panne, dtype: int64


In [6]:
# Paramètres des fenêtres
window_size = 100  # Taille de la fenêtre
step_size = 1     # Pas de glissement
numeric_columns = continuous_features

# Générer les fenêtres pour l'entraînement
train_windows = []
for i in tqdm(range(0, len(train_df) - window_size + 1, step_size)):
    window_data = train_df.iloc[i:i + window_size][numeric_columns].values.flatten()
    #panne_value = train_df.iloc[i + window_size - 1]["panne"]          # Dernière valeur de panne dans la fenêtre
    panne_value  = train_df.iloc[i:i + window_size]["panne"].mode()[0]  # Classe majoritaire
    train_windows.append({
        "index": i,
        "timestamp": train_df.iloc[i]["timestamp"],
        "sensor_data": window_data.tolist(),
        "panne": panne_value
    })

# Générer les fenêtres pour le test
test_windows = []
for i in tqdm(range(0, len(test_df) - window_size + 1, step_size)):
    window_data = test_df.iloc[i:i + window_size][numeric_columns].values.flatten()
    #panne_value  = test_df.iloc[i + window_size - 1]["panne"]           # Dernière valeur de panne dans la fenêtre
    panne_value   = test_df.iloc[i:i + window_size]["panne"].mode()[0]  # Classe majoritaire
    test_windows.append({
        "index": i,
        "timestamp": test_df.iloc[i]["timestamp"],
        "sensor_data": window_data.tolist(),
        "panne": panne_value
    })

print(f"Fenêtres d'entraînement : {len(train_windows)}")
print(f"Fenêtres de test : {len(test_windows)}")


100%|█████████████████████████████████████████████████████████████████████████| 329841/329841 [20:40<00:00, 265.94it/s]

Fenêtres d'entraînement : 1093762
Fenêtres de test : 329841


In [78]:
# Sauvegarder au format Parquet pour réduire la taille
train_df.to_parquet("train_windows_2025_01_18.parquet", compression="snappy", index=False)

# Charger le fichier Parquet
#train_df = pd.read_parquet("train_windows.parquet")


In [79]:
# Afficher le nombre de lignes
print(f"Nombre de lignes dans les données d'entraînement : {train_df.shape[0]}")


Nombre de lignes dans les données d'entraînement : 1093861


In [81]:
train_df2 = pd.read_parquet("train_windows_2025_01_18.parquet")

In [82]:
# Afficher le nombre de lignes
print(f"Nombre de lignes dans les données d'entraînement : {train_df2.shape[0]}")

Nombre de lignes dans les données d'entraînement : 1093861


In [84]:
import numpy as np

# Fonction pour calculer la distance euclidienne
def euclidean_distance(v1, v2):
    return np.sqrt(np.sum((np.array(v1) - np.array(v2)) ** 2))

# Rechercher les fenêtres similaires dans les données d'entraînement
def find_similar_windows(test_vector, train_df2, top_n=5):
    distances = []
    for _, row in train_df2.iterrows():
        distance = euclidean_distance(test_vector, row["sensor_data"])
        distances.append((distance, row["panne"]))
    
    # Trier par distance croissante
    distances.sort(key=lambda x: x[0])
    return distances[:top_n]  # Retourner les top_n fenêtres les plus proches


In [85]:
# Exemple de prédiction pour une fenêtre de test
test_vector = test_windows[0]["sensor_data"]  # Prenez un vecteur de test
similar_windows = find_similar_windows(test_vector, train_df)

# Prédiction par vote majoritaire
similar_pannes = [panne for _, panne in similar_windows]
predicted_panne = max(set(similar_pannes), key=similar_pannes.count)

print(f"Prédiction pour la fenêtre de test : {predicted_panne}")


Exception ignored in: <finalize object at 0x2c123cc9c60; dead>
Traceback (most recent call last):
  File "C:\Users\usermine\anaconda3\lib\weakref.py", line 591, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "C:\Users\usermine\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1176, in _close_pool_connections
    conn.close()
  File "C:\Users\usermine\anaconda3\lib\site-packages\urllib3\connection.py", line 322, in close
    super().close()
  File "C:\Users\usermine\anaconda3\lib\http\client.py", line 960, in close
    sock.close()   # close it manually... there may be other refs
  File "C:\Users\usermine\anaconda3\lib\socket.py", line 502, in close
    self._real_close()
  File "C:\Users\usermine\anaconda3\lib\ssl.py", line 1333, in _real_close
    super()._real_close()
  File "C:\Users\usermine\anaconda3\lib\socket.py", line 494, in _real_close
    def _real_close(self, _ss=_socket.socket):
KeyboardInterrupt: 


KeyError: 'sensor_data'

In [12]:
import kdbai_client as kdbai

# Connexion à KDB.AI
session = kdbai.Session(endpoint="https://cloud.kdb.ai/instance/atr0v2owym", api_key="a5b897a7b4-Av/lT//S3W++g8K11vVKfJo5rIj5gK492sbpts/1f+cr/5m5n5JluPcqcQR2gGiiQQ7SrCFAc7ma2RhT")
db = session.database('default')



In [ ]:

# Schéma de la table
schema = [
    {"name": "index", "type": "int64"},
    {"name": "timestamp", "type": "datetime64[ns]"},
    {"name": "sensor_data", "type": "float64s"},
    {"name": "panne", "type": "int64"}
]
indexes = [
    {"name": "flat_index", "type": "flat", "column": "sensor_data", "params": {"metric": "L2","dims": 500}}
]

# Création de la table
train_table = db.create_table("train_sensors", schema=schema, indexes=indexes)

# Insertion des données d'entraînement
batch_size = 100
for i in tqdm(range(0, len(train_windows), batch_size)):
    batch = train_windows[i:i + batch_size]
    train_table.insert(batch)

In [59]:
######################################################
#### Vérifier le Nombre d'Enregistrements Insérés ####
######################################################

aggs = {"row_count": ["count", "index"]}

query_result = train_table.query(aggs=aggs)

print(f"Nombre d'enregistrements insérés dans la table : {query_result['row_count']}")

Nombre d'enregistrements insérés dans la table : 0    947500
Name: row_count, dtype: int64


In [49]:
#######################################
##### Calculer le Point de Reprise ####
#######################################

# Récupérer le dernier index inséré
rows_inserted = query_result["row_count"].iloc[0]  # Nombre d'enregistrements déjà insérés
print(rows_inserted)

947500


In [39]:
################################
##### Reprendre l'Insertion ####
################################

# Définir la taille des batches
batch_size = 100

# Reprise de l'insertion à partir de rows_inserted
start_index = rows_inserted

print(f"Reprise de l'insertion à partir de l'index {start_index}")

for i in tqdm(range(start_index, len(train_windows), batch_size)):
    batch = train_windows[i:i + batch_size]
    try:
        train_table.insert(batch)
    except Exception as e:
        print(f"Erreur lors de l'insertion à l'index {i}: {e}")
        break


Reprise de l'insertion à partir de l'index 947500


  0%|                                                                                         | 0/1463 [00:03<?, ?it/s]

Erreur lors de l'insertion à l'index 947500: Neither Sparse nor Dense Index : /data/vdb/default/train_sensors/vdb/default_train_sensors_flat_index/metadata. OS reports: No such file or directory No index with this name present on disk


In [51]:
################################################################
#### Prédiction des Pannes pour l'Ensemble de Test Basée    ####
#### Sur le Vote Majoritaire dans l'Ensemble d'Entrainement ####
#### Avec la Classe Définie par la Dernière Valeur dans la  ####
#### Fenêtre                                                ####
################################################################

#################################################################################################################
### Distance Euclidienne (𝐿2)
### Les valeurs de nos vecteurs sont numériques ce qui convient parfaitement à la distance Euclidienne.    ######
### L'algorithme interprète les vecteurs comme des points dans un espace multidimensionnel, et la distance ######
### Euclidienne mesure la "proximité" géométrique entre ces points.
### Mesures Alternatives :
### Distance de Manhattan (L1), Distance de Hamming
##################################################################################################################


# Initialiser une liste pour stocker les résultats
results = []

# Parcourir toutes les fenêtres de test
for i, test_window in tqdm(enumerate(test_windows), total=len(test_windows)):
    # Récupérer le vecteur de test
    query_vector = test_window["sensor_data"]

    try:
        # Recherche vectorielle
        results_vector = train_table.search(
            vectors={"flat_index": [query_vector]},
            n=5  # Trouver les 5 motifs les plus similaires
        )

        # Extraire les valeurs de panne des motifs similaires
        similar_pannes = [result["panne"] for result in results_vector[0]]

        # Prédiction par vote majoritaire
        predicted_panne = max(set(similar_pannes), key=similar_pannes.count)

        # Ajouter les résultats à la liste
        results.append({
            "fenêtre": i,
            "prédiction": predicted_panne,
            "réelle": test_window['panne']  # Correspond à la dernière valeur dans la fenêtre de test
        })

    except Exception as e:
        print(f"Erreur lors de la prédiction pour la fenêtre {i}: {e}")

# Créer un DataFrame avec les résultats
results_df = pd.DataFrame(results)

# Calculer la précision globale
accuracy = (results_df["prédiction"] == results_df["réelle"]).mean()

# Compter les fenêtres par classe prédite
class_counts = results_df["prédiction"].value_counts()

# Afficher les résultats
print(f"Précision globale sur l'ensemble de test : {accuracy:.2%}")
print(f"Nombre total de fenêtres testées : {len(results_df)}")
print("\nPrédictions par classe :")
print(class_counts)

# Afficher le DataFrame
import ace_tools as tools; tools.display_dataframe_to_user(name="Résultats de Prédictions des Fenêtres", dataframe=results_df)


  0%|                                                                            | 2/329841 [00:00<14:51:22,  6.17it/s]

Erreur lors de la prédiction pour la fenêtre 0: Neither Sparse nor Dense Index : /data/vdb/default/train_sensors/vdb/default_train_sensors_flat_index/metadata. OS reports: No such file or directory No index with this name present on disk
Erreur lors de la prédiction pour la fenêtre 1: Neither Sparse nor Dense Index : /data/vdb/default/train_sensors/vdb/default_train_sensors_flat_index/metadata. OS reports: No such file or directory No index with this name present on disk


  0%|                                                                            | 4/329841 [00:00<13:41:35,  6.69it/s]

Erreur lors de la prédiction pour la fenêtre 2: Neither Sparse nor Dense Index : /data/vdb/default/train_sensors/vdb/default_train_sensors_flat_index/metadata. OS reports: No such file or directory No index with this name present on disk
Erreur lors de la prédiction pour la fenêtre 3: Neither Sparse nor Dense Index : /data/vdb/default/train_sensors/vdb/default_train_sensors_flat_index/metadata. OS reports: No such file or directory No index with this name present on disk


  0%|                                                                            | 6/329841 [00:00<13:16:05,  6.91it/s]

...

  0%|▏                                                                        | 1123/329841 [02:45<13:16:48,  6.88it/s]

Erreur lors de la prédiction pour la fenêtre 1121: Neither Sparse nor Dense Index : /data/vdb/default/train_sensors/vdb/default_train_sensors_flat_index/metadata. OS reports: No such file or directory No index with this name present on disk
Erreur lors de la prédiction pour la fenêtre 1122: Neither Sparse nor Dense Index : /data/vdb/default/train_sensors/vdb/default_train_sensors_flat_index/metadata. OS reports: No such file or directory No index with this name present on disk


  0%|▏                                                                        | 1124/329841 [02:46<13:29:35,  6.77it/s]

Erreur lors de la prédiction pour la fenêtre 1123: Neither Sparse nor Dense Index : /data/vdb/default/train_sensors/vdb/default_train_sensors_flat_index/metadata. OS reports: No such file or directory No index with this name present on disk


KeyboardInterrupt: 

In [53]:
# Vérifier si la table 'train_sensors' existe
try:
    table_exists = db.has_table("train_sensors")  # Vérifie l'existence de la table
    if table_exists:
        print("La table 'train_sensors' existe.")
    else:
        print("La table 'train_sensors' n'existe pas.")
except Exception as e:
    print(f"Erreur lors de la vérification de l'existence de la table : {e}")


Erreur lors de la vérification de l'existence de la table : 'Database' object has no attribute 'has_table'


In [54]:
# Afficher toutes les propriétés et méthodes de l'objet train_table
print(dir(train_table))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_table', 'default_result_type', 'drop', 'index', 'indexes', 'insert', 'name', 'query', 'refresh', 'schema', 'search', 'search_and_rerank', 'train', 'update_indexes']


In [55]:
# Récupérer les informations des index de la table
try:
    table_indexes = train_table.indexes
    print("Index de la table :", table_indexes)
except Exception as e:
    print(f"Erreur lors de la récupération des index : {e}")


Index de la table : [{'name': 'flat_index', 'type': 'flat', 'column': 'sensor_data', 'params': {'metric': 'L2', 'dims': 500}}]


In [58]:
# Récupérer le contenu ou les métadonnées du premier index
try:
    first_index = train_table.indexes[0]  # Premier index
    print("Propriétés du premier index :", first_index)
except Exception as e:
    print(f"Erreur lors de la récupération du premier index : {e}")


Propriétés du premier index : {'name': 'flat_index', 'type': 'flat', 'column': 'sensor_data', 'params': {'metric': 'L2', 'dims': 500}}


In [62]:
# Obtenir le schéma de la table
try:
    schema = train_table.schema
    print("Colonnes disponibles dans la table :")
    for column in schema:
        print(f"- {column['name']} (type : {column['type']})")
except Exception as e:
    print(f"Erreur lors de la récupération du schéma de la table : {e}")


Colonnes disponibles dans la table :
- index (type : int64)
- timestamp (type : datetime64[ns])
- sensor_data (type : float64s)
- panne (type : int64)


In [64]:
# Récupérer les premières lignes des colonnes disponibles
try:
    query_result = train_table.query(
        aggs={
            "first_index": ["first", "index"],
            "first_timestamp": ["first", "timestamp"],
            "first_sensor_data": ["first", "sensor_data"],
            "first_panne": ["first", "panne"]
        }
    )
    print("Premières lignes de la table :")
    print(f"Index : {query_result['first_index']}")
    print(f"Timestamp : {query_result['first_timestamp']}")
    print(f"Sensor Data : {query_result['first_sensor_data']}")
    print(f"Panne : {query_result['first_panne']}")
except Exception as e:
    print(f"Erreur lors de la récupération des données : {e}")


Premières lignes de la table :
Index : 0    0
Name: first_index, dtype: int64
Timestamp : 0   2020-02-02
Name: first_timestamp, dtype: datetime64[ns]
Sensor Data : 0    [1.391000000000001, 0.0376599999999999, 52.855...
Name: first_sensor_data, dtype: object
Panne : 0    0
Name: first_panne, dtype: int64


In [66]:
# Récupérer les deux derniers enregistrements par index
try:
    query_result = train_table.query(
        aggs={
            "last_two_indices": ["last", "index"],
            "last_two_timestamps": ["last", "timestamp"],
            "last_two_sensor_data": ["last", "sensor_data"],
            "last_two_pannes": ["last", "panne"]
        }
    )
    
    # Extraire l'avant-dernier enregistrement (le 2ème plus grand index)
    if len(query_result["last_two_indices"]) >= 2:
        print("Avant-dernier enregistrement :")
        print(f"Index : {query_result['last_two_indices'][-2]}")  # Avant-dernier index
        print(f"Timestamp : {query_result['last_two_timestamps'][-2]}")
        print(f"Sensor Data : {query_result['last_two_sensor_data'][-2]}")
        print(f"Panne : {query_result['last_two_pannes'][-2]}")
    else:
        print("Moins de deux enregistrements dans la table.")
except Exception as e:
    print(f"Erreur lors de la récupération de l'avant-dernier enregistrement : {e}")


Moins de deux enregistrements dans la table.


In [72]:
# Télécharger les premières lignes localement
try:
    query_result = train_table.query(
        aggs={
            "index": ["first", "index"],
            "timestamp": ["first", "timestamp"],
            "sensor_data": ["first", "sensor_data"],
            "panne": ["first", "panne"]
        }
    )

    # Convertir en DataFrame pour un traitement local
    import pandas as pd
    df = pd.DataFrame(query_result)

    # Accéder au 3ème enregistrement
    print("Troisième enregistrement localement :")
    print(df.iloc[2])  # 3ème ligne (index commence à 0)
except Exception as e:
    print(f"Erreur lors de la récupération des données : {e}")


Troisième enregistrement localement :
Erreur lors de la récupération des données : single positional indexer is out-of-bounds


In [77]:
import pandas as pd

# Récupérer les 10 premiers enregistrements de la table
try:
    query_result = train_table.query(
        aggs={
            "indices": ["first", "index"],
            "timestamps": ["first", "timestamp"],
            "sensor_data": ["first", "sensor_data"],
            "pannes": ["first", "panne"]
        }
    )

    # Vérifier si des données sont retournées
    if len(query_result["indices"]) >= 1:
        # Créer le DataFrame avec les 10 premiers enregistrements
        df2 = pd.DataFrame({
            "index": query_result["indices"][:1],
            "timestamp": query_result["timestamps"][:1],
            "sensor_data": query_result["sensor_data"][:1],
            "panne": query_result["pannes"][:1]
        })

        # Afficher le DataFrame
        print("Les 10 premiers enregistrements :")
        print(df2)
    else:
        print("La table contient moins de 10 enregistrements.")
except Exception as e:
    print(f"Erreur lors de la récupération des données : {e}")


Les 10 premiers enregistrements :
   index  timestamp                                        sensor_data  panne
0      0 2020-02-02  [1.391000000000001, 0.0376599999999999, 52.855...      0


In [73]:
# Vérifier le nombre total d'enregistrements
try:
    aggs = {"row_count": ["count", "index"]}
    query_result = train_table.query(aggs=aggs)
    print(f"Nombre total d'enregistrements dans la table : {query_result['row_count']}")
except Exception as e:
    print(f"Erreur lors de la vérification du nombre d'enregistrements : {e}")


Nombre total d'enregistrements dans la table : 0    947500
Name: row_count, dtype: int64


In [ ]:
################################################################
#### Prédiction des Pannes pour l'Ensemble de Test Basée    ####
#### Sur le Vote Majoritaire dans l'Ensemble d'Entrainement ####
################################################################
    
#################################################################################################################
### Distance Euclidienne (𝐿2)
### Les valeurs de nos vecteurs sont numériques ce qui convient parfaitement à la distance Euclidienne.    ######
### L'algorithme interprète les vecteurs comme des points dans un espace multidimensionnel, et la distance ######
### Euclidienne mesure la "proximité" géométrique entre ces points.
### Mesures Alternatives :
### Distance de Manhattan (L1), Distance de Hamming
##################################################################################################################


# Initialiser une liste pour stocker les résultats
results = []

# Parcourir toutes les fenêtres de test
for i, test_window in tqdm(enumerate(test_windows), total=len(test_windows)):
    # Récupérer le vecteur de test
    query_vector = test_window["sensor_data"]

    try:
        # Recherche vectorielle
        results_vector = train_table.search(
            vectors={"flat_index": [query_vector]},
            n=5  # Trouver les 5 motifs les plus similaires
        )

        # Extraire les valeurs de panne des motifs similaires
        similar_pannes = [result["panne"] for result in results_vector[0]]

        # Prédiction par vote majoritaire
        predicted_panne = max(set(similar_pannes), key=similar_pannes.count)

        # Ajouter les résultats à la liste
        results.append({
            "fenêtre": i,
            "prédiction": predicted_panne,
            "réelle": test_window['panne']
        })

    except Exception as e:
        print(f"Erreur lors de la prédiction pour la fenêtre {i}: {e}")

# Créer un DataFrame avec les résultats
results_df = pd.DataFrame(results)

# Calculer la précision globale
accuracy = (results_df["prédiction"] == results_df["réelle"]).mean()

# Compter les fenêtres par classe prédite
class_counts = results_df["prédiction"].value_counts()

# Afficher les résultats
print(f"Précision globale sur l'ensemble de test : {accuracy:.2%}")
print(f"Nombre total de fenêtres testées : {len(results_df)}")
print("\nPrédictions par classe :")
print(class_counts)

# Afficher le DataFrame
import ace_tools as tools; tools.display_dataframe_to_user(name="Résultats de Prédictions des Fenêtres", dataframe=results_df)



In [ ]:
################################################################
#### Prédiction des Pannes pour l'Ensemble de Test Basée    ####
#### Sur le Vote Majoritaire dans l'Ensemble d'Entrainement ####
################################################################

# Parcourir toutes les fenêtres de test
for i, test_window in enumerate(test_windows):
    # Récupérer le vecteur de test
    query_vector = test_window["sensor_data"]

    # Recherche vectorielle
    results = train_table.search(
        vectors={"flat_index": [query_vector]},
        n=5  # Trouver les 5 motifs les plus similaires
    )

    # Extraire les valeurs de panne des motifs similaires
    similar_pannes = results[0]["panne"].tolist()

    # Prédiction par majorité
    predicted_panne = max(set(similar_pannes), key=similar_pannes.count)

    # Affichage simple
    print(f"Fenêtre {i +} - Prédiction : {predicted_panne} - Valeur de Test : {test_window['panne']}")


In [ ]:
# Indice de la fenêtre 11 (les indices Python commencent à 0)
window_index = 10   # Fenêtre 622 correspond à l'index 621

# Récupérer les informations de la fenêtre
window_11 = test_windows[window_index]

# Affichage des détails
print(f"Fenêtre 11 : {window_11}")

In [8]:
##################################
###### SUPPRIMER LES TABLES ######
##################################
# Accéder aux tables existantes
train_table = db.table("train_sensors")

# Supprimer mes 2 tables
try:
    train_table.drop()
    print("Table 'train_sensors' supprimée avec succès.")
    print("Table 'test_sensors' supprimée avec succès.")
except Exception as e:
    print(f"Erreur lors de la suppression de 'train_sensors' : {e}")

Table 'train_sensors' supprimée avec succès.
Table 'test_sensors' supprimée avec succès.


<h3 style="text-align: center; font-family: Arial, sans-serif; color: BLUE;">################# QUELQUES VALIDATIONS #################</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [ ]:
print(f"Premiere fenetre train:\n{train_windows[0]}")

In [ ]:
aggs = {
    "row_count": ["count", "index"],
    "min_timestamp": ["min", "timestamp"],
    "max_timestamp": ["max", "timestamp"],
}

query_result = train_table.query(aggs=aggs)
print(f"Nombre de lignes : {query_result['row_count']}")
print(f"Timestamp minimum : {query_result['min_timestamp']}")
print(f"Timestamp maximum : {query_result['max_timestamp']}")

In [ ]:
# Comparer les timestamps entre train_df et KDB.ai
db_timestamps = pd.date_range(start="2020-04-17 21:30:00", end="2020-04-18 23:58:30", freq="10s")
missing_timestamps = train_df[~train_df["timestamp"].isin(db_timestamps)]

print(f"Lignes manquantes dans la base : {len(missing_timestamps)}")
print(missing_timestamps)


In [ ]:
# Préparer les lignes manquantes pour l'insertion
missing_rows = []
for row in missing_timestamps.itertuples(index=True, name="Row"):
    missing_rows.append({
        "index": row.Index,  # Utiliser l'index fourni par itertuples
        "timestamp": row.timestamp,
        "sensor_data": list(row[2:])  # Inclure les colonnes numériques sous forme de liste
    })

# Insérer les données manquantes dans la table train_sensors
batch_size = 10  # Taille du batch pour l'insertion
print(f"Réinsertion des {len(missing_rows)} lignes manquantes...")

for i in range(0, len(missing_rows), batch_size):
    batch = missing_rows[i:i + batch_size]
    try:
        train_table.insert(batch)
        print(f"Batch {i // batch_size + 1}/{-(-len(missing_rows) // batch_size)} inséré avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion du batch {i // batch_size + 1} : {e}")

print("Réinsertion des données manquantes terminée.")


In [ ]:
# Vérifiez les colonnes utilisées pour la création de sensor_data
numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]  # Exemple
print("Colonnes utilisées pour sensor_data :", numeric_columns)

# Préparer les lignes manquantes pour l'insertion
missing_rows = []
for row in missing_timestamps.itertuples(index=True, name="Row"):
    # Vérifiez les données avant de les ajouter
    try:
        sensor_data = [getattr(row, col) for col in numeric_columns]  # Extraire les colonnes nécessaires
        missing_rows.append({
            "index": row.Index,  # Utiliser l'index fourni par itertuples
            "timestamp": row.timestamp,
            "sensor_data": sensor_data  # Liste de données numériques
        })
    except AttributeError as e:
        print(f"Erreur lors de la préparation des données pour l'index {row.Index} : {e}")

# Réinsertion dans la base
batch_size = 10
print(f"Réinsertion des {len(missing_rows)} lignes manquantes...")

for i in range(0, len(missing_rows), batch_size):
    batch = missing_rows[i:i + batch_size]
    try:
        train_table.insert(batch)
        print(f"Batch {i // batch_size + 1}/{-(-len(missing_rows) // batch_size)} inséré avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion du batch {i // batch_size + 1} : {e}")

print("Réinsertion des données manquantes terminée.")


In [ ]:
# Vérification des dimensions de chaque vecteur avant l'insertion
expected_length = 500  # Longueur attendue des vecteurs

invalid_vectors = []
for row in missing_rows:
    if len(row["sensor_data"]) != expected_length:
        invalid_vectors.append(row)

# Si des vecteurs invalides sont trouvés, les afficher
if invalid_vectors:
    print(f"Vecteurs invalides détectés : {len(invalid_vectors)}")
    for idx, invalid_row in enumerate(invalid_vectors[:5]):  # Afficher les 5 premiers vecteurs invalides
        print(f"Index : {invalid_row['index']}, Longueur : {len(invalid_row['sensor_data'])}")
else:
    print("Tous les vecteurs ont la bonne dimension.")

# Réinsertion uniquement des vecteurs valides
valid_rows = [row for row in missing_rows if len(row["sensor_data"]) == expected_length]

print(f"Réinsertion des {len(valid_rows)} lignes valides...")
for i in range(0, len(valid_rows), batch_size):
    batch = valid_rows[i:i + batch_size]
    try:
        train_table.insert(batch)
        print(f"Batch {i // batch_size + 1}/{-(-len(valid_rows) // batch_size)} inséré avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion du batch {i // batch_size + 1} : {e}")

print("Réinsertion des données valides terminée.")


In [ ]:
# Taille réelle de la première fenêtre
print(f"Taille des données dans la première fenêtre d'entraînement : {len(train_windows[0]['sensor_data'])}")
print(f"Taille des données dans la première fenêtre de test : {len(test_windows[0]['sensor_data'])}")

# Cela correspond à window_size (100) × nombre de colonnes (numeric_columns, soit 5).

print('--------------------------------')
# Vérifier la taille des fenêtres générées
for i in range(5):  # Tester sur les 5 premières fenêtres
    actual_window = train_df.iloc[i:i + window_size][numeric_columns]
    print(f"Fenêtre {i} - Taille réelle : {len(actual_window)}")
    

In [ ]:
# Initialiser des compteurs pour détecter les erreurs
invalid_index_count = 0
invalid_timestamp_count = 0
invalid_sensor_data_count = 0

# Parcourir toutes les fenêtres pour vérifier les types et formats
for i, window in enumerate(train_windows):
    # Vérifier l'index
    if not isinstance(window['index'], int):
        print(f"Fenêtre {i} : 'index' non valide ({type(window['index'])})")
        invalid_index_count += 1
    
    # Vérifier le timestamp
    try:
        pd.Timestamp(window['timestamp'])  # Vérifie si le timestamp est convertible
    except Exception as e:
        print(f"Fenêtre {i} : 'timestamp' non valide ({window['timestamp']})")
        invalid_timestamp_count += 1
    
    # Vérifier sensor_data
    if not isinstance(window['sensor_data'], list) or len(window['sensor_data']) != 500:
        print(f"Fenêtre {i} : 'sensor_data' non valide (Longueur : {len(window['sensor_data'])})")
        invalid_sensor_data_count += 1
    elif not all(isinstance(value, float) for value in window['sensor_data']):
        print(f"Fenêtre {i} : 'sensor_data' contient des valeurs non flottantes.")
        invalid_sensor_data_count += 1

# Résumé des erreurs détectées
print(f"Erreurs dans 'index' : {invalid_index_count}")
print(f"Erreurs dans 'timestamp' : {invalid_timestamp_count}")
print(f"Erreurs dans 'sensor_data' : {invalid_sensor_data_count}")

In [ ]:
# Tester l'insertion avec un petit lot

# Schéma pour les tables
sensor_schema = [
    {"name": "index", "type": "int64"},               # Identifiant unique
    {"name": "timestamp", "type": "datetime64[ns]"}, # Timestamp
    {"name": "sensor_data", "type": "float64s"}      # Données vectorielles
]

# Index pour la recherche vectorielle
indexes = [
    {
        "name": "flat_index",
        "type": "flat",
        "column": "sensor_data",
        "params": {"metric": "L2","dims": 500}
    }
]

# Créer les tables
train_table = db.create_table("train_sensors", schema=sensor_schema, indexes=indexes)
test_table = db.create_table("test_sensors", schema=sensor_schema, indexes=indexes)


single_window = [train_windows[0]]  # Utiliser uniquement la première fenêtre
try:
    train_table.insert(single_window)
    print("Insertion réussie pour une seule fenêtre.")
except Exception as e:
    print(f"Erreur lors de l'insertion d'une seule fenêtre : {e}")

In [ ]:
from datetime import datetime

try:
    datetime.fromisoformat(single_window[0]['timestamp'])
    print("Le format du timestamp est correct.")
except ValueError as e:
    print(f"Problème avec le format du timestamp : {e}")


In [ ]:
for value in single_window[0]['sensor_data']:
    if not isinstance(value, float):
        print(f"Problème détecté : {value} de type {type(value)}")


In [ ]:
if len(single_window[0]['sensor_data']) != 500:
    print("La longueur de sensor_data n'est pas correcte.")


In [ ]:
print("Schéma de la table :")
print(train_table.schema)


In [ ]:
from datetime import datetime

for window in train_windows:
    window['timestamp'] = datetime.fromisoformat(window['timestamp'])


In [ ]:
train_table.query(limit=5)  # Affiche les 5 premières lignes


In [ ]:
test_row_count = test_table.query(aggs=aggs)
print(f"Lignes stockées dans la table de test : {test_row_count}")


In [ ]:
# Afficher le contenu du lot envoyé pour inspection
print("Contenu du lot de test :")
for i, window in enumerate(test_batch):
    print(f"Fenêtre {i}: {window}")


In [ ]:
# Vérification approfondie des valeurs
for i, window in enumerate(test_batch):
    if not all(isinstance(v, float) and not (np.isnan(v) or np.isinf(v)) for v in window['sensor_data']):
        print(f"Fenêtre {i} contient des valeurs invalides.")


In [ ]:
# Remplacer les valeurs invalides par une valeur par défaut (ex. : 0.0)
for window in train_windows:
    window['sensor_data'] = [
        0.0 if (np.isnan(v) or np.isinf(v)) else v
        for v in window['sensor_data']
    ]


In [ ]:
# Vérifier la structure des 5 premières fenêtres
for window in train_windows[:5]:
    print(f"Index : {window['index']}")
    print(f"Timestamp : {window['timestamp']}")
    print(f"Longueur de sensor_data : {len(window['sensor_data'])}")
    print(f"Exemple de données : {window['sensor_data'][:5]}")


In [ ]:
# Vérification des longueurs des fenêtres
for i, window in enumerate(train_windows):
    if len(window['sensor_data']) != window_size * len(numeric_columns):
        print(f"Fenêtre {i} a une taille incorrecte : {len(window['sensor_data'])}")
print("Validation de la longueur des fenêtres terminée.")

In [ ]:
# Vérification de la continuité temporelle
for i, window in enumerate(train_windows):
    timestamps = train_df.iloc[window['index']:window['index'] + window_size]['timestamp']
    time_diffs = timestamps.diff().dt.total_seconds().dropna()
    if not all(time_diffs == 10):  # Vérifie si tous les gaps sont de 10 secondes
        print(f"Fenêtre {i} a des timestamps non consécutifs.")
print("Validation de la continuité temporelle terminée.")

In [ ]:
# Exemple : Vérifier la réponse brute
response = train_table.insert(train_windows[:5])
print("Réponse brute :", response)

In [ ]:
print(db.tables)

In [ ]:
# Requête pour compter les lignes dans une table
train_row_count = db.query("SELECT COUNT(*) AS row_count FROM train_sensors").iloc[0]['row_count']
print(f"Lignes stockées dans la table d'entraînement : {train_row_count}")

test_row_count = db.query("SELECT COUNT(*) AS row_count FROM test_sensors").iloc[0]['row_count']
print(f"Lignes stockées dans la table de test : {test_row_count}")


In [ ]:
# Afficher toutes les méthodes et attributs de l'objet db
print(dir(db.tables))

In [ ]:
# Afficher toutes les méthodes et attributs de l'objet train_table
print(dir(train_table))

In [ ]:
# Obtenir les détails sur la méthode query
help(train_table.query)

In [ ]:
# Définir l'agrégation pour compter les lignes
aggs = {'row_count': ['count', 'index']}  # Compte le nombre de lignes en utilisant la colonne 'index'

# Effectuer la requête pour compter les lignes
train_row_count = train_table.query(aggs=aggs)
print(f"Lignes stockées dans la table d'entraînement : {train_row_count}")


In [ ]:
# Parametres des fenetres temporelles
window_size = 100  # Taille de la fenetre
step_size = 1      # Pas de glissement
numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]

# Générer les fenetres pour l'entrainement
train_windows = [
    {
        "index": i,
        "timestamp": train_df.iloc[i]["timestamp"],
        "sensor_data": train_df.iloc[i:i + window_size][numeric_columns].values.flatten().tolist()
    }
    for i in range(0, len(train_df) - window_size + 1, step_size)
]

# Générer les fenetres pour le test
test_windows = [
    {
        "index": i,
        "timestamp": test_df.iloc[i]["timestamp"],
        "sensor_data": test_df.iloc[i:i + window_size][numeric_columns].values.flatten().tolist()
    }
    for i in range(0, len(test_df) - window_size + 1, step_size)
]

print(f"Fenêtres d'entraînement : {len(train_windows)}")
print(f"Fenêtres de test : {len(test_windows)}")


In [ ]:
from sklearn.decomposition import PCA

numeric_columns = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]

# Construire les vecteurs pour chaque fenetre (numeric_columns)
window_vectors = np.array([
    window[numeric_columns].values.flatten() for window in windows
])

# Appliquer PCA pour réduire la dimensionnalité
pca = PCA(n_components=8)  # Réduire a 8 dimensions
compressed_vectors = pca.fit_transform(window_vectors)

print(f"Dimensions des vecteurs compressés : {compressed_vectors.shape}")
print(f"Vecteur compressé pour la première fenêtre : {compressed_vectors[0]}")